In [1]:
# !pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

### Only if you have a GPU

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

# check if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5



### End of GPU config section

In [4]:
# see available tfds data sets
", ".join(tfds.list_builders())

'abstract_reasoning, aeslc, aflw2k3d, amazon_us_reviews, arc, bair_robot_pushing_small, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, cmaterdb, cnn_dailymail, coco, coil100, colorectal_histology, colorectal_histology_large, cos_e, curated_breast_imaging_ddsm, cycle_gan, deep_weeds, definite_pronoun_resolution, diabetic_retinopathy_detection, div2k, dmlab, downsampled_imagenet, dsprites, dtd, duke_ultrasound, dummy_dataset_shared_generator, dummy_mnist, emnist, eraser_multi_rc, esnli, eurosat, fashion_mnist, flic, flores, food101, gap, gigaword, glue, groove, higgs, horses_or_humans, i_naturalist2017, image_label_folder, imagenet2012, imagenet2012_corrupted, imagenet_resized, imagenette, imagewang, imdb_reviews, iris, kitti, kmnist, lfw,

In [5]:
# using TFDS dataset
# note: as_supervised converts dicts to tuples
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", 
                                with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", 
                      as_supervised=True)

In [6]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [7]:
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [8]:
# Use the default tokenizer settings
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

In [9]:
imdb_encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, 
                                                   tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


In [10]:
# Lets verify tokenization and encoding works
for example, label in imdb_train.take(1):
    print(example)
    encoded = imdb_encoder.encode(example.numpy())
    print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [11]:
# transformation fucntions to be used with the dataset
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post', 
                                 maxlen=150)
    return np.array(pad[0], dtype=np.int64)  


def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, 
                                       inp=[sample], 
                                       Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label

In [12]:
# test the transformation on a small subset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)

In [13]:
for review, label in tst.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))

tf.Tensor(
[17742 56635 49898  9860 43863 69648 88571  2913 74960 65318 59132 43610
 54077 90053 62861 53649  1572 49300 92378 40939 10306 14306 14152 13803
 17115 74960 51007 20831 81120 59132 86909 46281 51007 40939 26463 81363
 75536 76259 14152 69648  8531 12165  2659 17742 69648 30986 49898 28902
 33424 16784 89115 23845 20418 88861 78870 92944 65227 33128 17915 64804
 13263 20702 65227  8126 51007 88793 40982 11237 53775 41392 22350 39116
  1873 34111 32416  8552 34986 74425 51911 90053 56635 78267 14306 59822
 78870 17297 73762 59132 59822 69648 75561 56635 37594 47467 34379  4485
 36749 81188 77632 38691 75561 76586 92378 74083 70330 14152 27519 60108
  8531 70330 54077 90053  8531 10595 24278 81188 81363 22821 82725 93731
 80234     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [14]:
#now tokenize/encode/pad all training
# and testing data
encoded_train = imdb_train.map(encode_tf_fn,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn,
                             num_parallel_calls=tf.data.experimental.AUTOTUNE)

## Preparing the model

In [15]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=100

In [16]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model


In [17]:
model = build_model_lstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (100, None, 64)           6011584   
_________________________________________________________________
lstm (LSTM)                  (100, 64)                 33024     
_________________________________________________________________
dense (Dense)                (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

## Model Training and Evaluation

In [19]:
# Prefetch for performance
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [20]:
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 23s 93ms/step - loss: 0.4175 - accuracy: 0.8011 - Precision: 0.7806 - Recall: 0.8378
Epoch 2/10
250/250 [==============================] - 21s 85ms/step - loss: 0.1829 - accuracy: 0.9344 - Precision: 0.9346 - Recall: 0.9342
Epoch 3/10
250/250 [==============================] - 21s 84ms/step - loss: 0.1060 - accuracy: 0.9650 - Precision: 0.9659 - Recall: 0.9641
Epoch 4/10
250/250 [==============================] - 21s 84ms/step - loss: 0.0746 - accuracy: 0.9756 - Precision: 0.9764 - Recall: 0.9748
Epoch 5/10
250/250 [==============================] - 21s 85ms/step - loss: 0.0631 - accuracy: 0.9796 - Precision: 0.9815 - Recall: 0.9776
Epoch 6/10
250/250 [==============================] - 21s 84ms/step - loss: 0.0450 - accuracy: 0.9860 - Precision: 0.9852 - Recall: 0.9868
Epoch 7/10
250/250 [==============================] - 21s 84ms/step - loss: 0.0441 - accuracy: 0.9857 - Precision: 0.9859 - Recall: 0.9854
Epoch 8/10
250/250 [=======

In [21]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

    250/Unknown - 20s 79ms/step - loss: 0.7937 - accuracy: 0.8232 - Precision: 0.8852 - Recall: 0.7428

[0.7936793555021286, 0.82324, 0.8852131, 0.7428]

# BiLSTM Model

In [51]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=50

In [52]:
dropout=0.5
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.5)),
    #tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
    #tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [53]:
bilstm = build_model_bilstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

bilstm.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (50, None, 128)           12023168  
_________________________________________________________________
bidirectional_14 (Bidirectio (50, None, 128)           98816     
_________________________________________________________________
bidirectional_15 (Bidirectio (50, 128)                 98816     
_________________________________________________________________
dense_8 (Dense)              (50, 1)                   129       
Total params: 12,220,929
Trainable params: 12,220,929
Non-trainable params: 0
_________________________________________________________________


In [54]:
bilstm.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [55]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [56]:
bilstm.fit(encoded_train_batched, epochs=10)

Epoch 1/10
500/500 [==============================] - 72s 144ms/step - loss: 0.3734 - accuracy: 0.8278 - Precision: 0.8213 - Recall: 0.8378
Epoch 2/10
500/500 [==============================] - 63s 126ms/step - loss: 0.1497 - accuracy: 0.9443 - Precision: 0.9444 - Recall: 0.9442
Epoch 3/10
500/500 [==============================] - 63s 126ms/step - loss: 0.0814 - accuracy: 0.9718 - Precision: 0.9724 - Recall: 0.9711
Epoch 4/10
500/500 [==============================] - 64s 127ms/step - loss: 0.0510 - accuracy: 0.9822 - Precision: 0.9824 - Recall: 0.9821
Epoch 5/10
500/500 [==============================] - 63s 126ms/step - loss: 0.0263 - accuracy: 0.9904 - Precision: 0.9909 - Recall: 0.9900
Epoch 6/10
500/500 [==============================] - 91s 182ms/step - loss: 0.0226 - accuracy: 0.9921 - Precision: 0.9921 - Recall: 0.9922
Epoch 7/10
500/500 [==============================] - 103s 205ms/step - loss: 0.0192 - accuracy: 0.9934 - Precision: 0.9934 - Recall: 0.9934
Epoch 8/10
500/500 

In [57]:
bilstm.evaluate(encoded_test.batch(BATCH_SIZE))

    500/Unknown - 20s 39ms/step - loss: 1.0473 - accuracy: 0.8355 - Precision: 0.8025 - Recall: 0.8899

[1.0472896423190832, 0.83548, 0.8025395, 0.88992]